## Importing libraries

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Open training data

In [0]:
link = 'https://drive.google.com/open?id=1f8gIhhWgbp4nYQ72VpW5HFUFjZspRMD-'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_training = pd.read_csv('Filename.csv')

In [0]:
df_training

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


# Multi-index by case concept name and event concept name

In [0]:
df_training.set_index(['case concept:name', 'event concept:name'], inplace=True)
df_training.sort_index(inplace=True)

In [0]:
df_training

eventID   ...     event time:timestamp
case concept:name event concept:name                  ...                         
173688            A_ACCEPTED                       5  ...  01-10-2011 11:42:43.308
                  A_ACTIVATED                     24  ...  13-10-2011 10:37:29.226
                  A_APPROVED                      22  ...  13-10-2011 10:37:29.226
                  A_FINALIZED                      7  ...  01-10-2011 11:45:09.243
                  A_PARTLYSUBMITTED                1  ...  01-10-2011 00:38:44.880
...                                              ...  ...                      ...
206318            W_Afhandelen leads  44955422687235  ...  03-02-2012 17:49:29.108
                  W_Afhandelen leads  44955422687237  ...  03-02-2012 17:56:00.998
206321            A_DECLINED          44959717654530  ...  03-02-2012 17:09:19.112
                  A_PARTLYSUBMITTED   44959717654529  ...  03-02-2012 17:08:39.459
                  A_SUBMITTED         44959717654528  ...  03-02-2012 17:08:39.200

[214377 rows x 5 columns]

Example of a single case

In [0]:
df_training.loc[173688]

,eventID,case REG_DATE,case AMOUNT_REQ,event lifecycle:transition,event time:timestamp
event concept:name,,,,,
A_ACCEPTED,5,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 11:42:43.308
A_ACTIVATED,24,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_APPROVED,22,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_FINALIZED,7,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 11:45:09.243
A_PARTLYSUBMITTED,1,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:38:44.880
A_PREACCEPTED,2,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:39:37.906
A_REGISTERED,21,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_SUBMITTED,0,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:38:44.546
O_ACCEPTED,23,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226


Another (smaller) example

In [0]:
df_training.loc[206318]

,eventID,case REG_DATE,case AMOUNT_REQ,event lifecycle:transition,event time:timestamp
event concept:name,,,,,
A_DECLINED,44955422687236,2012-02-03T17:07:38.334+01:00,5000,COMPLETE,03-02-2012 17:55:57.294
A_PARTLYSUBMITTED,44955422687233,2012-02-03T17:07:38.334+01:00,5000,COMPLETE,03-02-2012 17:07:38.843
A_SUBMITTED,44955422687232,2012-02-03T17:07:38.334+01:00,5000,COMPLETE,03-02-2012 17:07:38.334
W_Afhandelen leads,44955422687234,2012-02-03T17:07:38.334+01:00,5000,SCHEDULE,03-02-2012 17:07:52.820
W_Afhandelen leads,44955422687235,2012-02-03T17:07:38.334+01:00,5000,START,03-02-2012 17:49:29.108
W_Afhandelen leads,44955422687237,2012-02-03T17:07:38.334+01:00,5000,COMPLETE,03-02-2012 17:56:00.998


# Split in training and validation set

###1. indices = df.index.levels[0] takes all the unique from level=0 index - which are all the cases
###2. train_indices = np.random.choice(indices,size=int(len(indices)*0.80), replace=False) samples 80% of the indices chosen in previous step - which basically is a 80/20 split of the training data and validation data
###3. Finally we split train and validation accordingly

In [0]:
#1
indices = df_training.index.levels[0]

#2
train_indices = np.random.choice(indices,size=int(len(indices)*0.80), replace=False)
validation_indices = np.setdiff1d(indices, train_indices)

#3
train = df_training[np.in1d(df_training.index.get_level_values(0), train_indices)]
validation = df_training[np.in1d(df_training.index.get_level_values(0), validation_indices)]

Check whether the same case is not in both the training and validation set

In [0]:
train.loc[173688]

,eventID,case REG_DATE,case AMOUNT_REQ,event lifecycle:transition,event time:timestamp
event concept:name,,,,,
A_ACCEPTED,5,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 11:42:43.308
A_ACTIVATED,24,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_APPROVED,22,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_FINALIZED,7,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 11:45:09.243
A_PARTLYSUBMITTED,1,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:38:44.880
A_PREACCEPTED,2,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:39:37.906
A_REGISTERED,21,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226
A_SUBMITTED,0,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,01-10-2011 00:38:44.546
O_ACCEPTED,23,2011-10-01T00:38:44.546+02:00,20000,COMPLETE,13-10-2011 10:37:29.226


In [0]:
validation.loc[173688]

KeyError: ignored

## The length of both dataframes should add up to 214377

In [0]:
len(train)+len(validation)

214377

# Function to split every dataframe into training and validation (according 80/20 split)

In [0]:
def split_data(df1, column1, column2):
  df1.set_index([column1, column2], inplace=True)
  df1.sort_index(inplace=True)

  #1
  indices = df1.index.levels[0]

  #2
  train_indices = np.random.choice(indices,size=int(len(indices)*0.80), replace=False)
  validation_indices = np.setdiff1d(indices, train_indices)

  #3
  train = df1[np.in1d(df1.index.get_level_values(0), train_indices)]
  validation = df1[np.in1d(df1.index.get_level_values(0), validation_indices)]

  return train, validation



Try with a different dataset

In [0]:
link = 'https://drive.google.com/open?id=1wNw7ozOZTP_CJHGmIALZF1ssAHPrQ_ms'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_training2 = pd.read_csv('Filename.csv')

In [0]:
df_training2 = df_training2.iloc[0:250000]

In [0]:
train2, validation2 = split_data(df_training2, 'case concept:name', 'event concept:name')

In [0]:
train2

eventID   ...     event time:timestamp
case concept:name      event concept:name                      ...                         
Application_1000339879 A_Accepted              25713969201159  ...  18-03-2016 09:25:20.591
                       A_Complete              25713969201166  ...  18-03-2016 09:31:53.211
                       A_Concept               25713969201157  ...  17-03-2016 13:58:21.623
                       A_Create Application    25713969201152  ...  17-03-2016 13:57:10.159
                       A_Incomplete            25713969201183  ...  28-03-2016 13:35:50.445
...                                                       ...  ...                      ...
Application_999507989  W_Handle leads          25602300051459  ...  17-03-2016 07:54:40.461
                       W_Validate application  25602300051476  ...  23-03-2016 10:05:10.324
                       W_Validate application  25602300051477  ...  23-03-2016 10:05:10.327
                       W_Validate application  25602300051480  ...  23-03-2016 10:12:03.011
                       W_Validate application  25602300051481  ...  23-03-2016 14:18:21.202

[200362 rows x 10 columns]

In [0]:
validation2

eventID   ...     event time:timestamp
case concept:name      event concept:name                      ...                         
Application_1002996676 A_Accepted              16196321673223  ...  22-02-2016 14:51:01.329
                       A_Cancelled             16196321673234  ...  24-03-2016 08:00:19.559
                       A_Complete              16196321673230  ...  22-02-2016 14:54:09.638
                       A_Concept               16196321673221  ...  19-02-2016 15:07:53.518
                       A_Create Application    16196321673216  ...  19-02-2016 15:07:16.599
...                                                       ...  ...                      ...
Application_998600110  W_Validate application  13361643257893  ...  02-03-2016 14:02:09.376
                       W_Validate application  13361643257894  ...  04-03-2016 09:04:34.203
                       W_Validate application  13361643257895  ...  04-03-2016 09:04:39.577
                       W_Validate application  13361643257896  ...  04-03-2016 12:59:48.338
                       W_Validate application  13361643257899  ...  04-03-2016 13:34:09.768

[49638 rows x 10 columns]